In [93]:
# import scipy.stats
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# 
# plt.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['axes.unicode_minus'] = False
# 
# data = pd.read_excel("附件.xlsx",index_col=0,usecols="A:O")
# # data = pd.DataFrame(data,columns=['CBTS','EPDS','HADS','infant_behavioral']) 
# data = data[:-20]
# 
# correlation_matrix = data.corr(method='spearman')
# print(correlation_matrix)
# 
# plt.figure(figsize=(10,8))
# sns.heatmap(correlation_matrix,annot=True,cmap='coolwarm',linewidths=0.5)
# plt.title('Spearman Correlation Heatmap')
# 
# CBTS = np.array(data['CBTS'])
# EPDS = np.array(data['EPDS'])
# HADS = np.array(data['HADS'])
# ib = np.array(data['infant_behavioral'])
# 
# s_CBTS = scipy.stats.spearmanr(CBTS,ib)
# s_EPDS = scipy.stats.spearmanr(EPDS,ib)
# s_HADS = scipy.stats.spearmanr(HADS,ib)
# 
# p_CBTS = scipy.stats.pearsonr(CBTS,ib)
# p_EPDS = scipy.stats.pearsonr(EPDS,ib)
# p_HADS = scipy.stats.pearsonr(HADS,ib)
# 
# print(s_CBTS)
# print(s_EPDS)
# print(s_HADS)
# print('--------------------------------')
# print(p_CBTS)
# print(p_EPDS)
# print(p_HADS)

In [94]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

In [95]:
def TimeConversion(s):
    return s.hour + s.minute / 60 + s.second / 3600

datas = pd.read_excel("附件.xlsx", index_col=0, usecols="A:O")
datas['婴儿行为特征'] = datas['婴儿行为特征'].map({'中等型': 1, '安静型': 2, '矛盾型': 3})
datas.drop(datas[datas['整晚睡眠时间（时：分：秒）']=='99:99'].index,inplace=True)
UnknownData=datas[-20:]
datas=datas[:-20]
datas['整晚睡眠时间（时：分：秒）']=datas['整晚睡眠时间（时：分：秒）'].map(TimeConversion)
datas

,母亲年龄,婚姻状况,教育程度,妊娠时间（周数）,分娩方式,CBTS,EPDS,HADS,婴儿行为特征,婴儿性别,婴儿年龄（月）,整晚睡眠时间（时：分：秒）,睡醒次数,入睡方式
编号,,,,,,,,,,,,,,
1,34,2,5,37.0,1,3,13,9,1.0,1,1,10.0,3.0,2.0
2,33,2,5,42.0,1,0,0,3,2.0,2,3,11.0,0.0,4.0
3,37,2,5,41.0,1,4,8,9,2.0,1,1,12.0,1.0,2.0
4,31,2,5,37.5,1,6,16,13,2.0,2,3,11.0,2.0,1.0
5,36,1,5,40.0,1,1,3,3,1.0,2,3,10.5,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,27,2,5,42.0,1,12,20,14,1.0,1,1,9.0,1.0,2.0
387,31,2,3,39.0,1,4,7,12,1.0,1,2,11.0,2.0,4.0
388,31,2,5,41.6,1,0,3,1,2.0,1,1,10.5,2.0,1.0


In [96]:

# 处理数据，找出特征值和目标值
x = datas[['母亲年龄', '婚姻状况', '教育程度', '妊娠时间（周数）', '分娩方式', 'CBTS', 'EPDS',
           'HADS', '婴儿性别', '婴儿年龄（月）']]

y = datas[['婴儿行为特征']]
# print(x)

# 分割数据集到训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# 进行处理(特征工程)
dict = DictVectorizer(sparse=False)
print(x_train.info())
x_train = dict.fit_transform(x_train.to_dict(orient="records"))

# print(dict.get_feature_names_out())

x_test = dict.transform(x_test.to_dict(orient="records"))


<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 158 to 190
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   母亲年龄      291 non-null    int64  
 1   婚姻状况      291 non-null    int64  
 2   教育程度      291 non-null    int64  
 3   妊娠时间（周数）  291 non-null    float64
 4   分娩方式      291 non-null    int64  
 5   CBTS      291 non-null    int64  
 6   EPDS      291 non-null    int64  
 7   HADS      291 non-null    int64  
 8   婴儿性别      291 non-null    int64  
 9   婴儿年龄（月）   291 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 25.0 KB
None


In [97]:
# 用决策树进行预测
dec = DecisionTreeClassifier()

dec.fit(x_train, y_train)

# 预测准确率
print("预测的准确率为：", dec.score(x_test, y_test))

# 导出决策树的结构
export_graphviz(dec, out_file="./tree.dot")

# feature_names=['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


预测的准确率为： 0.4489795918367347


In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import graphviz
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz

warnings.filterwarnings('ignore')

plt.figure(figsize=(10,6),dpi=300)

df = pd.read_csv('./C_Python.csv')

def fun(value,lb,ub,M):
    if value < lb:
        return 1.0-(lb-value)/M
    elif value > ub:
        return 1.0-(value-ub)/M
    else:
        return 1.0

def fun2(value,sum):
    value = value / sum
    if(value==0):
        return 0
    else:
        return np.log(value) * value

    # TOPSIS 
class TOPSIS():
    def __init__(self,data:pd.DataFrame):
        self.data = data
        # 母亲最佳生育年龄 
        self.ma_lb = 25
        self.ma_ub = 29
        self.ma_M = max(self.ma_lb
        self.data['mother_age'].min(),self.data['mother_age'].max()-self.ma_ub)

        # 母亲最佳妊娠时间 
        self.pd_lb = 38
        self.pd_ub = 40
        self.pd_M = max(self.pd_lb
        self.data['pregnancy_duration'].min(),self.data['pregnancy_duration'].max()
        self.pd_ub)

        # 婴儿最佳睡眠时间 
        self.st_lb = 10
        self.st_ub = 12
        self.st_M = max(self.st_lb
        self.data['sleeping_time'].min(),self.data['sleeping_time'].max()-self.st_ub)

        # 数据预处理 
        def process(self):
            # 删除id 
            self.data = self.data.drop(columns='id')

            # 对全部异常值的处理 
            # 婚姻状态大于2 
            self.data =
    self.data.drop(self.data[self.data['marriage_status']>2].index)
    # 填充后20行空缺值 
    self.data = self.data.fillna(value=0)
    # 睡眠时间超过正常值 
    self.data = self.data.drop(self.data[(self.data.sleeping_time<1) &
                                         (self.data.sleeping_way>0)].index)

    # 区间型指标的正向化处理 
    # 母亲生育年龄 
    self.data['mother_age'] = self.data['mother_age'].apply(lambda
                                                                x:fun(x,self.ma_lb,self.ma_ub,self.ma_M))

    # 母亲妊娠时间 
    self.data['pregnancy_duration'] =
self.data['pregnancy_duration'].apply(lambda
                                          x:fun(x,self.pd_lb,self.pd_ub,self.pd_M))

# # 独热编码 
# # 婚姻状态 
# dummies = 
pd.get_dummies(self.data['marriage_status'],prefix='marriage_status')
# self.data = self.data.drop(columns='marriage_status') 
# self.data = pd.concat([self.data,dummies],axis=1) 

# # 婴儿性别 
# dummies = 
pd.get_dummies(self.data['infant_gender'],prefix='infant_gender')
# self.data = self.data.drop(columns='infant_gender') 
# self.data = pd.concat([self.data,dummies],axis=1) 

# # 受教育程度 
# dummies = 
pd.get_dummies(self.data['educational_level'],prefix='educational_level')
# self.data = self.data.drop(columns='educational_level') 
# self.data = pd.concat([self.data,dummies],axis=1) 

# # 分娩方式 
# dummies = 
pd.get_dummies(self.data['delivery_way'],prefix='delivery_way')
# self.data = self.data.drop(columns='delivery_way') 
# self.data = pd.concat([self.data,dummies],axis=1) 

# # 婴儿年龄 
# dummies = 
pd.get_dummies(self.data['infant_age'],prefix='infant_age')
# self.data = self.data.drop(columns='infant_age') 
# self.data = pd.concat([self.data,dummies],axis=1) 

# # 睡眠方式 
# dummies = 
pd.get_dummies(self.data['sleeping_way'],prefix='sleeping_way')
# self.data = self.data.drop(columns='sleeping_way') 
# self.data = pd.concat([self.data,dummies],axis=1) 

# 心理指标归一化 
self.data['CBTS'] = self.data['CBTS'].apply(lambda x:(30-x)/30)
self.data['EPDS'] = self.data['EPDS'].apply(lambda x:(30-x)/30)
self.data['HADS'] = self.data['HADS'].apply(lambda x:(30-x)/30)

return self.data

def tocsv(self,coloum_name:str):
    res = np.array(self.data[coloum_name])
    res = res.reshape((-1,1))
    res = pd.DataFrame(res)
    res.to_csv('./C_'+coloum_name+'.csv',index=False)

TOPSIS_data = TOPSIS(df)
data = TOPSIS_data.process()
data_known = data[:-20]
data_predict = data[-20:]

feature_names = data_known[[
    'mother_age',
    'pregnancy_duration',
    'marriage_status',
    'educational_level',
    'CBTS',
    'EPDS',
    'HADS']]
label_name = data_known['infant_behavioral']
X_var = feature_names.values
Y_var = label_name.values

x_train,x_test,y_train,y_test =
train_test_split(X_var,Y_var,test_size=0.2,random_state=0)

# 使用决策树 
dtc_model = DecisionTreeClassifier(criterion='entropy',max_depth=6)
dtc_model.fit(x_train,y_train)
y_test_pred = dtc_model.predict(x_test)
print('Accuracy of the dtc_model is {:.0%}'.format(accuracy_score(y_test,
                                                                  y_test_pred)))
print(classification_report(y_test,y_test_pred))

# 尝试使用其他模型 
# xgboost 
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
y_train_xgb = np.array(y_train-1)
y_test_xgb = np.array(y_test-1)
xgb_model.fit(x_train,y_train_xgb)
y_test_pred = xgb_model.predict(x_test)

print('Accuracy of the xgb_model is {:.0%}'.format(accuracy_score(y_test_xgb,
                                                                  y_test_pred)))
print(classification_report(y_test_xgb,y_test_pred))

# # 尝试xgboost网格搜索 
# # 定义参数的取值范围 
# learning_rate = [0.01,0.015,0.025,0.05,0.1] 
# subsample = [0.5,0.6,0.7,0.8,0.9] 
# colsample_bytree = [0.5,0.6,0.7,0.8,0.9] 
# max_depth = [3,5,6,7,9] 

# parameters = {'learning_rate':learning_rate, 
#               'subsample':subsample, 
#               'colsample_bytree':colsample_bytree, 
#               'max_depth':max_depth} 
# xgb_model = XGBClassifier() 

# # 进行网格搜索 
# xgb_model_GS = 
GridSearchCV(xgb_model,parameters,scoring='accuracy',verbose=1,n_jobs=-1)
# xgb_model_GS = xgb_model_GS.fit(x_train,y_train_xgb) 
# print(xgb_model_GS.best_params_) 
# print(xgb_model_GS.best_score_) 
# print(xgb_model_GS.best_estimator_) 

# xgb_model_new = 
XGBClassifier(colsample_bytree=0.5,learning_rate=0.01,max_depth=3,subsample=0.
5)
# xgb_model_new.fit(x_train,y_train_xgb) 

# # 使用数据预测类别 
# y_train_pred = xgb_model_new.predict(x_train) 
# y_test_pred = xgb_model_new.predict(x_test) 

# print('Accuracy of the optimized xgb_model in training set is 
{:.0%}'.format(accuracy_score(y_train_xgb, y_train_pred)))
# print('Accuracy of the optimized xgb_model in testing set is 
{:.0%}'.format(accuracy_score(y_train_xgb, y_train_pred)))

# 随机森林 
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier()
rfc_model.fit(x_train,y_train)
y_test_pred = rfc_model.predict(x_test)
print('Accuracy of the rfc_model is {:.0%}'.format(accuracy_score(y_test,
                                                                  y_test_pred)))
print(classification_report(y_test,y_test_pred))

# ET 
from sklearn.ensemble import ExtraTreesClassifier
et_model = ExtraTreesClassifier()
et_model.fit(x_train,y_train)
y_test_pred = et_model.predict(x_test)
print('Accuracy of the et_model is {:.0%}'.format(accuracy_score(y_test,
                                                                 y_test_pred)))
print(classification_report(y_test,y_test_pred))

# 朴素贝叶斯 
from sklearn.naive_bayes import GaussianNB
gnb_model = GaussianNB()
gnb_model.fit(x_train,y_train)
y_test_pred = gnb_model.predict(x_test)
print('Accuracy of the gnb_model is {:.0%}'.format(accuracy_score(y_test,
                                                                  y_test_pred)))
print(classification_report(y_test,y_test_pred))

# K近邻 
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
knn_model.fit(x_train,y_train)
y_test_pred = knn_model.predict(x_test)
print('Accuracy of the knn_model is {:.0%}'.format(accuracy_score(y_test,
                                                                  y_test_pred)))
print(classification_report(y_test,y_test_pred))

# 逻辑回归 
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train,y_train)
y_test_pred = lr_model.predict(x_test)
print('Accuracy of the lr_model is {:.0%}'.format(accuracy_score(y_test,
                                                                 y_test_pred)))
print(classification_report(y_test,y_test_pred))

# 支持向量机 
from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(x_train,y_train)
y_test_pred = svc_model.predict(x_test)
print('Accuracy of the svc_model is {:.0%}'.format(accuracy_score(y_test,
                                                                  y_test_pred)))
print(classification_report(y_test,y_test_pred))

# 决策树节点信息输出成txt文本 
output_list = plot_tree(dtc_model,feature_names=feature_names.columns)
file = open('C_nodes_2.txt','wt',-1)
file.write(str(output_list))
file.close()

# 决策树节点信息输出成png图片 
dot_data = export_graphviz(dtc_model,
                           feature_names=feature_names.columns,
                           out_file=None,
                           node_ids=True,
                           rounded=True,
                           filled=True,
                           special_characters=True)
dot=graphviz.Source(dot_data,filename='C_DecisionTree_2.gv',format='png')
dot.view()

# 模型预测 
feature_names = data_predict[[
    'mother_age',
    'pregnancy_duration',
    'marriage_status',
    'educational_level',
    'CBTS',
    'EPDS',
    'HADS']]

X_var = feature_names.values
Y_var = data_predict['infant_behavioral'].values
predict = dtc_model.predict(X_var)
print('The prediction result using the decision tree model is: ')
print(predict)

# 预测结果保存为csv文件 
predict = predict.reshape((-1,1))
df_predict = pd.DataFrame(predict)
df_predict.to_csv('./C_prediction_2.csv',index=False,header=None) 